In [ ]:
import json
import time
from selenium.webdriver.remote.webdriver import By
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support import expected_conditions as EC
import subprocess

class gptParser:
    def __init__(self, driver, gpt_url: str = 'https://chatgpt.com/?model=gpt-4o'):
        """ ChatGPT parser
        Args:
            driver_path (str, optional): The path of the chromedriver.
            gpt_url (str, optional): The url of ChatGPT.
        """
        # Start a webdriver instance and open ChatGPT
        self.driver = driver
        self.driver.get(gpt_url)
        self.prompt_textarea = None
        self.upload_button = None
        self.current_response_id = None
        
        # login_button = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.CSS_SELECTOR, 'button[data-testid="login-button"]'))
        login_button = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.CSS_SELECTOR, 'button.btn.relative.btn-secondary.btn-small'))
        login_button.click()

        google_login_button = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.CSS_SELECTOR, ".social-btn:nth-child(1)"))
        google_login_button.click()
        
        google_login_email_input = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.ID, "identifierId"))
        google_login_email_input.send_keys("xxxxxxxxx@gmail.com")
        google_login_email_input.send_keys(Keys.ENTER)

        time.sleep(2)

        google_login_password_input = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.CSS_SELECTOR, 'input[type="password"][name="Passwd"]'))
        google_login_password_input.send_keys("**********")
        google_login_password_input.send_keys(Keys.ENTER)

        self.search_elements()

    @staticmethod
    def get_driver(driver_path: str = None,):
        return uc.Chrome() if driver_path is None else uc.Chrome(driver_path)

    def search_elements(self):
        self.prompt_textarea = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.CSS_SELECTOR, 'p[data-placeholder="傳訊息給 ChatGPT"]'))
        self.upload_button = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.CSS_SELECTOR, 'button[aria-label="附加檔案"]'))
        self.new_chat_button = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.CSS_SELECTOR, 'button[aria-label="新交談"]'))

    def prompt(self, msg: str):
        self.search_elements()
        for m in msg.split("\n"):
            self.prompt_textarea.send_keys(m)
            self.prompt_textarea.send_keys(Keys.SHIFT, Keys.ENTER)

    def get_send_button(self):
        send_button = WebDriverWait(self.driver, 60).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="傳送提示詞"]')))
        return send_button

    def enter(self):
        send_button = WebDriverWait(self.driver, 60).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="傳送提示詞"]')))
        send_button.click()

    def read_response(self):
        # 當讀到分割符號，代表新的回應已經完全產生，及可取得最新的回應
        time.sleep(3)
        seperator = WebDriverWait(self.driver, 60).until(lambda x: x.find_element(By.CSS_SELECTOR, 'div.mt-3.w-full.empty\\:hidden'))        
        if seperator:
            current_response = WebDriverWait(self.driver, 60).until(lambda x: x.find_elements(By.CSS_SELECTOR, 'div[data-message-author-role="assistant"]'))[-1]
        
            if self.current_response_id == current_response.get_attribute("data-message-id"):
                print("等待新回應中...")
                return self.read_response()

            self.current_response_id = current_response.get_attribute("data-message-id")
            print("最新回應 ID: " + current_response.get_attribute("data-message-id"))
            print("最新回應: " + current_response.text)
            return current_response.text

    def new_chat(self):
        self.search_elements()
        self.new_chat_button.click()
    
    def upload_document(self, file_path: str):
        self.search_elements()
        self.upload_button.click()
        upload_from_computer_button = WebDriverWait(self.driver, 60).until(lambda x: x.find_elements(By.CSS_SELECTOR, 'div[role="menuitem"]')[-1])
        upload_from_computer_button.click()

        # 使用 AppleScript 激活包含該標題的 Google Chrome 窗口
        cmd = f'''
        osascript -e 'tell application "Google Chrome"
            set windowList to every window
            repeat with aWindow in windowList
                if title of aWindow contains "{self.driver.title}" then
                    set index of aWindow to 1
                    activate
                    exit repeat
                end if
            end repeat
        end tell'
        '''
        subprocess.call(cmd, shell=True)

        # 按下 Shift + Command + G 打開 "前往文件夾"
        cmd = f'''
        osascript -e 'tell application "System Events"
            keystroke "G" using {{shift down, command down}}
            delay 1
            keystroke "{file_path}"
            delay 1
            keystroke return
            delay 1
            keystroke return
        end tell'
        '''
        subprocess.call(cmd, shell=True)

    def init_gpt_analysis(self):
        self.new_chat()
        self.upload_document("/Users/uncleben006/nextgen/code/python/ptt_stock_vane_api/config/data/stock_dataset.csv")
        self.upload_document("/Users/uncleben006/nextgen/code/python/ptt_stock_vane_api/config/data/stock_targets.csv")
        self.prompt("""You are an AI specialized in generating JSON responses. Please do not provide any content other than the JSON response (including calculation processes, thoughts, reasoning, etc.). You will analyze comments on the Taiwan PTT forum's stock section to identify the companies mentioned in each comment and determine the market sentiment of each comment.

        First, to help you better identify the companies mentioned in the comments, I will provide a dataset call "stock_dataset.csv". This dataset records the names, stock codes, business descriptions of all Taiwanese listed companies, as well as the nicknames given by PTT forum members and the sources of these nicknames. Please read and remember the content of this dataset carefully, as it will be the basis for identifying the companies mentioned in the comments.

        Next, I will provide you "stock_targets.csv" with the names and stock codes of listed companies in Taiwan. When you identify any terms or nicknames in the comments that refer to a company, I need you to use this data to cross-reference and tell me which company is mentioned in the comment and the stock code of that company. For example, if a comment mentions "台G" or "螃蟹", I would like the final answer to be {"台積電":"2330"} or {"瑞昱":"2379"}. The point is that the company name and stock code must match without any errors.

        Please read each comment thoroughly, carefully examining whether it mentions any stock codes, nicknames, or company names from "stock_dataset.csv". Do not just glance over and assume there is no relevant information; the devil is in the details. Comes to details, be particularly attentive when comments mention numbers, as people in the forum like to refer to company using stock code. For example, 2498 refers to HTC(宏達國際電子), and 00900 refers to a popular ETF in Taiwan, 富邦特選高股息30. In addition, you should also stay sensitive to various terms, even common ones, cause people in the forum often use nicknames to refer to companies. For example, "肉鬆要噴發了！" refers to "廣達" There are many similar examples, and I've recorded them in the stock_dataset.csv file.

        Regarding market sentiment, please define sentiment within a range of -1 to 1. The more bearish the sentiment, the closer the value should be to -1; the more bullish the sentiment, the closer the value should be to 1. Similarly, please thoroughly understand each comment, as they directly reflect the current sentiment of investors. Do not attempt to use formulas or write programs to understand these sentiments. Carefully interpret and try to grasp the meaning behind each message, and be sure to provide the most accurate market sentiment score.

        Each comment needs to output a JSON object. I will give you multiple comments each time(about 5 to 10), so you must always output an array of JSON objects, with each object containing a "stock_sentiment" key with a floating-point value and a "stock_targets" key with an array value.

        The sample format is as follows:
        [
            {"comment": "台G跟發哥漲翻了我看", "stock_sentiment": 0.7732953298, "stock_targets": [{"台積電":"2498"}, {"聯發科":"2454"}]},
            {"comment": "我看我要去睡公園了我", "stock_sentiment": -0.325342319, "stock_targets": [""]},
            {"comment": "這邊追國統還不如買兆聯，國統每天當沖比較適合", "stock_sentiment": -0.2982719372, "stock_targets": [{"國統":"8936"}, {"兆聯":"6944"}]}
        ]
        
        Please ensure that the number of your JSON array output matches the number of input comments. Even if a comment consists of only a punctuation mark, it should be treated as a separate comment. This is very important, as even punctuation or spaces might carry sentiment.

        Please return the JSON in plain text, without displaying it in code format and without line breaks to avoid difficulties in copying.
        """)
        self.enter()
    
    def ask_gpt(self, comment_batch_content):
        try:
            self.prompt(json.dumps(comment_batch_content, ensure_ascii=False).replace('\\"', "'"))
            time.sleep(1)
            send_button = self.get_send_button() # 有時候送出按鈕會莫名一直無法按
            if send_button:
                send_button.click()
                response_text = self.read_response()
                results = json.loads(response_text)
                if len(results) != len(comment_batch_content):
                    print("結果與留言數量不符，即將開啟新聊天視窗詢問相同內容。")
                    time.sleep(5)
                    self.init_gpt_analysis()
                    return self.ask_gpt(comment_batch_content)
            return results
        except Exception as e:
            # 代表 GPT-4o 回傳的不是 JSON 格式或有其他問題產生，要導正太麻煩了，直接重開對話繼續
            # 唯一例外是當達到 rate limit 時，需要等待半小時才能繼續使用
            if self.check_rate_limit():
                return 'reach_rate_limit'
            else:
                print(f"發生錯誤: {e}")
                print("即將開啟新聊天視窗並詢問相同內容")
                time.sleep(5)
                self.init_gpt_analysis()
                return self.ask_gpt(comment_batch_content)
        
    def check_rate_limit(self):
        try:
            WebDriverWait(self.driver, 1).until(lambda x: x.find_element(By.CSS_SELECTOR, 'div.font-bold.text-token-text-primary'))
            return True
        except:
            return False

    def close(self):
        self.driver.quit()

In [2]:
driver = gptParser.get_driver()
gpt_parser = gptParser(driver)

In [3]:
import psycopg2

# 建立連線
conn = psycopg2.connect(
    user="root",
    password="root",
    host="localhost",
    port="5433",
    database="postgres"
)

with conn.cursor() as cur:

    sql = """SELECT * FROM public.ptt_stock_paragraphs
                WHERE paragraph_published_time BETWEEN '2024-08-01' AND '2024-08-07' 
                AND paragraph_tag NOT IN ('新聞','請益','創作','心得','閒聊')
                AND paragraph_title NOT LIKE '%創作%'
                AND paragraph_content NOT LIKE '%水桶%'
                AND done_analyze = False"""

    cur.execute(sql)
    records = cur.fetchall()

    # 每 7 個留言一批，傳進 GPT-4o 分析，避免一次傳太多留言造成 GPT-4o 失焦
    batch_size = 7
    for record in records:
        success = False
        attempts = 0
        comments = record[8]
        comments_analyze_results = []

        # 初始化 gpt prompt
        gpt_parser.init_gpt_analysis()

        # 將留言分批傳入 GPT-4o 分析
        for i in range(0, len(comments), batch_size):

            comment_batch = comments[i:i+batch_size]
            comment_batch_content = [comment["comment"] for comment in comment_batch]

            # 若這批已經分析過，則印出後跳過
            if all("stock_sentiment" in comment and "stock_targets" in comment for comment in comment_batch):
                comment_batch_results = [{"comment": comment["comment"], 
                                  "stock_sentiment": comment["stock_sentiment"], 
                                  "stock_targets": comment["stock_targets"]} 
                                  for comment in comment_batch]
                comments_analyze_results += comment_batch_results
                print(comment_batch_results)
                print(f"文章:{record[0]} {record[3]}")
                print(f"總更新留言數: {len(comments_analyze_results)}")
                print(f"留言已分析，跳過")
                print("=============================================")
                continue

            results = gpt_parser.ask_gpt(comment_batch_content)

            if results == 'reach_rate_limit': # 重新初始化 GPT-4o
                print("已達 rate limit，將關閉此瀏覽器，等待兩個半小時後再次嘗試")
                time.sleep(5)
                gpt_parser.close()
                time.sleep(10800)
                driver = gptParser.get_driver()
                gpt_parser = gptParser(driver)
                gpt_parser.init_gpt_analysis()
                # redo current batch
                i -= batch_size
                continue
            
            comments_analyze_results += results
            
            # 更新 comments
            for k, comment in enumerate(comments[i:i+batch_size]):
                comments[i+k]["stock_sentiment"] = results[k]["stock_sentiment"]
                comments[i+k]["stock_targets"] = [] if results[k]["stock_targets"] == [""] else comments_analyze_results[i+k]["stock_targets"]
            
            # 更新資料庫
            sql = "UPDATE public.ptt_stock_paragraphs SET comments = %s WHERE id = %s"
            cur.execute(sql, (json.dumps(comments, ensure_ascii=False), record[0]))
            conn.commit()
            print(f"更新文章:{record[0]} {record[3]}")
            print(f"總更新留言數: {len(comments_analyze_results)}")
            print("=============================================")
        
        # 若 comments 全部都有 stock_sentiment 和 stock_targets，則將 done_analyze 設為 True
        if all("stock_sentiment" in comment and "stock_targets" in comment for comment in comments):
            sql = "UPDATE public.ptt_stock_paragraphs SET done_analyze = %s WHERE id = %s"
            cur.execute(sql, (True, record[0]))
            conn.commit()
            print(f"文章:{record[0]} {record[3]}")
            print("已完成分析")
            print("=============================================")


[{'comment': ' 噴', 'stock_sentiment': 0.621, 'stock_targets': []}, {'comment': ' 3x-0.06這樣不噴說不過去吧', 'stock_sentiment': -0.142, 'stock_targets': []}, {'comment': ' 什麼時候回441 I’m waiting for u  QQ', 'stock_sentiment': -0.346, 'stock_targets': []}, {'comment': ' 蒸丸！', 'stock_sentiment': 0.221, 'stock_targets': []}, {'comment': ' 釣魚概念股', 'stock_sentiment': 0.101, 'stock_targets': []}, {'comment': ' 推文小心', 'stock_sentiment': -0.112, 'stock_targets': []}, {'comment': ' 現在是神獸雲豹的時代惹~', 'stock_sentiment': 0.534, 'stock_targets': [{'雲豹能源': '6829'}]}]
文章:152691 [情報] 6547 高端疫苗 113年H1: -0.18
總更新留言數: 7
留言已分析，跳過
[{'comment': ' 賠錢，噴', 'stock_sentiment': -0.452, 'stock_targets': []}, {'comment': ' -0.18*2*2024,有fu糗', 'stock_sentiment': -0.213, 'stock_targets': []}, {'comment': ' 怎樣都會是0.06的倍數 好神奇', 'stock_sentiment': 0.103, 'stock_targets': []}, {'comment': ' 噴到500', 'stock_sentiment': 0.712, 'stock_targets': []}, {'comment': ' 綠色顧眼睛', 'stock_sentiment': 0.022, 'stock_targets': []}, {'comment': ' 雲豹一代目',

KeyboardInterrupt: 